In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('./')
from planar_ising.planar_ising.inference_and_sampling import InferenceAndSampling
from planar_ising.planar_ising.planar_ising_model_generator import PlanarIsingModelGenerator
from planar_ising.planar_graph.planar_graph_edges import PlanarGraphEdges
from planar_ising.planar_graph.planar_graph_generator import PlanarGraphGenerator

from triangular_lattice_v4 import *
from mpmath import matrix, mp, log, power, fadd, fmul, fdiv, mpf, zeros

%load_ext autoreload
%autoreload 2


In [ ]:
# precision
mp.dps = 150

In [ ]:
def partition_function_method(sets,G,double_measurements):
    q=mpf('2')
    beta = log((q**2+1)/q)

    jmatrix, E0 = model_couplings_high_prec(sets,G,double_measurements,tp = 'num')
    jmatrix_planar = matrix(-beta*jmatrix)
    ising_model = PlanarIsingModelGenerator.generate_triang_ising_model(database, 
                                                                        jmatrix_planar)
    ising_inference = InferenceAndSampling(ising_model)
    ising_inference.prepare()
    log_partition_function = ising_inference.compute_logpf()
    Snum  = -log_partition_function+0*beta*E0-np.log(0.5)
    print('E0:', E0)
    print('beta*E0:', beta*E0)

    jmatrix,E0 = model_couplings_high_prec(sets,G,double_measurements,tp = 'denom') 
    jmatrix_planar = matrix(-beta*jmatrix)
    print('beta*E0:', beta*E0)
    ising_model = PlanarIsingModelGenerator.generate_triang_ising_model(database, 
                                                                        jmatrix_planar)
    ising_inference = InferenceAndSampling(ising_model)
    ising_inference.prepare()
    log_partition_function = ising_inference.compute_logpf()
    Sden  = -log_partition_function+0*beta*E0-np.log(0.5)
    return Snum-Sden

In [ ]:
np.random.seed(1)
Lph,T = 32,32
L = int(Lph/2)+(int(Lph/2)%2)
print('L', L)
p = 1-(int(Lph/2)%2)
# measurement probability

prob=0.
G=20

#=================================================================================

basisx,basisy,sets,measured,sole_measurements,double_measurements =\
trnglr_lattice(L,Lph,T,p,prob=prob)
adjmatrix = allOR(sets)
database = list_edges(adjmatrix,basisx,basisy)  

##=================================================================================

S1 = cellular_automaton_method(Lph,T,measured,smp_std=100000)
S2 = partition_function_method(sets,G,double_measurements)

print('CA method:',S1)
print('PF method:',S2)